In [2]:
import fecfile
import pandas as pd
import os
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from dateutil.parser import parse
from IPython.display import display
import geopandas as gpd
import us

Time the notebook

In [3]:
start = datetime.datetime.now()

Define logging function for writing output

In [4]:
def log(obj, description):
    print(description + ":")
    with open("../output/log.txt", "a") as f:
        f.write(f"{description}:\n\n{obj}\n\n---\n")
    return obj

Set some viewing options

In [5]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 500)

Read dataframe of filings

In [6]:
filings = (
    pd.read_csv("../data/filings.csv")
)

filings.head(3)

,committee_id,filing_id,cycle,form_type,date_filed,date_coverage_to,date_coverage_from,report_title,report_period,fec_uri,paper,amended,amended_uri,is_amendment,original_filing,original_uri,committee_type,contributions_total,cash_on_hand,disbursements_total,receipts_total
0,C00401224,1190211,2016,F3,2017-11-03,2016-11-28,2016-10-20,POST-GENERAL,PG,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1190211/,False,False,NaN,True,1132265.0,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1132265/,W,69306148.22,24023158.64,70473625.32,70884967.84
1,C00401224,1167570,2016,F3,2017-06-27,2016-10-19,2016-10-01,PRE-GENERAL,PREG,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1167570/,False,False,NaN,True,1118321.0,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1118321/,W,38988748.30,23611816.12,36653226.26,41086481.96
2,C00401224,1166534,2016,F3,2017-06-19,2016-12-31,2016-11-29,YEAR-END,YE,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1166534/,False,False,NaN,True,1144458.0,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1144458/,W,14665549.11,24071137.23,15378810.13,15426788.72


Get only the monthlies

In [7]:
midyear = (
    filings
    .loc[
        lambda x: x["report_title"].str.contains("MID-YEAR", na = False)
    ]
    # remove filings that have been superceded by subsequent filings
    .loc[
        lambda x: x['amended'] == False
    ]
    [["filing_id",
        "date_filed",
        "date_coverage_to",
        "date_coverage_from",
        "report_title",
        "report_period",
        "fec_uri",
         "cycle"
     ]]
)

midyear.head(3)

,filing_id,date_filed,date_coverage_to,date_coverage_from,report_title,report_period,fec_uri,cycle
28,1034155,2015-11-23,2015-06-30,2015-06-23,MID-YEAR,NaN,http://docquery.fec.gov/cgi-bin/dcdev/forms/C00401224/1034155/,2016


Load candidates file

In [8]:
candidates = pd.read_csv(
    "../data/candidates.csv"
)

Read dataframe of all actblue contributions

In [9]:
actblue = pd.read_csv(
    "../output/merged_actblue.csv",
    dtype = {
        "filing_id": "object",
        "contributor_organization_name": "object"
    },
    parse_dates = ['contribution_date',]
)

actblue.head(3)

,entity_type,filer_committee_id_number,filing_id,transaction_id,contribution_date,contribution_amount,contribution_aggregate,contributor_organization_name,contributor_first_name,contributor_last_name,contributor_street_1,contributor_street_2,contributor_state,contributor_zip_code,contributor_state.1,contributor_employer,contributor_occupation,contribution_purpose_descrip,memo_text_description,donor_id,committee_id,candidate_name,latest_contribution_aggregate
0,IND,C00401224,1034155,SA11AI_28202360,2015-06-29,5.0,35.0,NaN,HOMA,A. HASHEMI,22 CLINTON AVE 9,NaN,CT,06901,CT,NOT EMPLOYED,NOT EMPLOYED,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),HOMA|A HASHEMI|06901,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,35.0
1,IND,C00401224,1034155,SA11AI_28249728,2015-06-30,5.0,35.0,NaN,HOMA,A. HASHEMI,22 CLINTON AVE 9,NaN,CT,06901,CT,NOT EMPLOYED,NOT EMPLOYED,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),HOMA|A HASHEMI|06901,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,35.0
2,IND,C00401224,1034155,SA11AI_28108486,2015-06-27,25.0,25.0,NaN,SHERI,A. OLSON,4008 SW ARROYO CT,NaN,WA,98146,WA,SELF,ARCHITECT,Earmark,Earmarked for DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE (C00000935),SHERI|A OLSON|98146,C00000935,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,25.0


Read zip code populations from census

In [241]:
county_pop = (
    pd
    .read_csv(
        "../data/census/ACS_17_5YR_B01003_COUNTY/ACS_17_5YR_B01003_with_ann.csv",
        encoding = "ISO-8859-1",
        skiprows = 1, 
        dtype = {
            "Id2": str,
            "Id": str,
            "Estimate; Total": int
        }
    )
)

place_names = county_pop['Geography'].str.split(",", expand = True)
county_pop['county_name'] = place_names[0]

county_pop.head(3)

,Id,Id2,Geography,Estimate; Total,Margin of Error; Total,county_name
0,0500000US01001,01001,"Autauga County, Alabama",55036,*****,Autauga County
1,0500000US01003,01003,"Baldwin County, Alabama",203360,*****,Baldwin County
2,0500000US01005,01005,"Barbour County, Alabama",26201,*****,Barbour County


Make a lookup table of fips to county

In [267]:
to_county = dict(
    zip(
        county_pop['Id2'], 
        county_pop["county_name"])
)

to_state = dict(
    zip(
        county_pop["Id2"],
        county_pop["Geography"]
    )
)

to_county

{'01001': 'Autauga County',
 '01003': 'Baldwin County',
 '01005': 'Barbour County',
 '01007': 'Bibb County',
 '01009': 'Blount County',
 '01011': 'Bullock County',
 '01013': 'Butler County',
 '01015': 'Calhoun County',
 '01017': 'Chambers County',
 '01019': 'Cherokee County',
 '01021': 'Chilton County',
 '01023': 'Choctaw County',
 '01025': 'Clarke County',
 '01027': 'Clay County',
 '01029': 'Cleburne County',
 '01031': 'Coffee County',
 '01033': 'Colbert County',
 '01035': 'Conecuh County',
 '01037': 'Coosa County',
 '01039': 'Covington County',
 '01041': 'Crenshaw County',
 '01043': 'Cullman County',
 '01045': 'Dale County',
 '01047': 'Dallas County',
 '01049': 'DeKalb County',
 '01051': 'Elmore County',
 '01053': 'Escambia County',
 '01055': 'Etowah County',
 '01057': 'Fayette County',
 '01059': 'Franklin County',
 '01061': 'Geneva County',
 '01063': 'Greene County',
 '01065': 'Hale County',
 '01067': 'Henry County',
 '01069': 'Houston County',
 '01071': 'Jackson County',
 '01073': 

Read zcta to county crosswalk from census

In [276]:
zcta_census = (
    pd
    .read_csv(
        "../data/census/zcta_county.csv", 
        dtype = { 
            "ZCTA5": str, 
            "STATE": str,
            "GEOID": str
        }
    )
)

zcta_census.head()

,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,ZAREA,ZAREALAND,COPOP,COHU,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,00601,72,1,72001,18465,7695,165132671,164333375,18570,7744,167459085,166659789,19483,8125,173777444,172725651,99.43,99.37,98.61,98.60,94.77,94.71,95.03,95.14
1,00601,72,141,72141,105,49,2326414,2326414,18570,7744,167459085,166659789,33149,14192,298027589,294039825,0.57,0.63,1.39,1.40,0.32,0.35,0.78,0.79
2,00602,72,3,72003,41520,18073,83734431,79288158,41520,18073,83734431,79288158,41959,18258,117948080,79904246,100.00,100.00,100.00,100.00,98.95,98.99,70.99,99.23
3,00603,72,5,72005,54689,25653,82063867,81880442,54689,25653,82063867,81880442,60949,28430,195741178,94608641,100.00,100.00,100.00,100.00,89.73,90.23,41.92,86.55
4,00606,72,93,72093,6276,2740,94864349,94851862,6615,2877,109592548,109580061,6276,2740,94864349,94851862,94.88,95.24,86.56,86.56,100.00,100.00,100.00,100.00


Some ZIP codes cross state boundaries. The code below finds the main state, by population, associated with any given ZIP Code Tabulation Area.

In [293]:
zcta_states = (
    zcta_census
    .groupby([ "ZCTA5", "STATE"])
    ["ZPOPPCT"]
    .sum()
    .reset_index()
    .sort_values("ZPOPPCT", ascending=False)
    .groupby("ZCTA5")
    ["STATE"]
    .first() # Get the top sorted (largest)
    .apply(
        lambda x: us.states.lookup(x).name
    )
    .to_frame("state")
    .reset_index()
    .rename(
        columns = { "ZCTA5": "contributor_zip_code"} # for easier merging later
    )
)

zcta_states.head()

,contributor_zip_code,state
0,00601,Puerto Rico
1,00602,Puerto Rico
2,00603,Puerto Rico
3,00606,Puerto Rico
4,00610,Puerto Rico


ZIP codes also cross state boundaries. The code below finds the main state, by population, associated with any given ZIP Code Tabulation Area.

In [294]:
def find_county_name(geoId):
    try:
        return county_lookup[geoId]
    except:
        return ""

zcta_counties = (
    zcta_census
    .groupby([ "ZCTA5", "GEOID"]) # County fips code
    ["COPOPPCT"]
    .sum()
    .reset_index()
    .sort_values("COPOPPCT", ascending=False)
    .groupby("ZCTA5")
    ["GEOID"]
    .first() 
    .apply(
        find_county_name
    )
    .to_frame("county_name")
    .reset_index()
    .rename(
        columns = { "ZCTA5": "contributor_zip_code"} # for easier merging later
    )
)

zcta_counties.head()

,contributor_zip_code,county_name
0,00601,
1,00602,
2,00603,
3,00606,
4,00610,


Build a DataFrame of contributors, states and population

In [303]:
# Get number of donors per candidate per zip
zip_donors = (
    actblue
    .groupby(
        ["candidate_name", 
         "contributor_zip_code"]
    )
    .size()
    .to_frame('contributors')
    .reset_index()
)

# Get the state names and zip population using the census dfs
zip_totals = (
    zip_donors
    .merge(
        zcta_states,
        on = "contributor_zip_code",
        how = "left"
    )
    .merge(
        zcta_counties,
        on = "contributor_zip_code",
        how = "left"
    )
    .merge(
        zcta_census[["ZCTA5", "ZPOP", "GEOID"]],
        left_on = "contributor_zip_code",
        right_on = "ZCTA5"
    )
    .drop( columns = "ZCTA5" )
    .rename(
        columns = {"GEOID": "county_fip", "ZPOP": "zip_population"}
    )
)

zip_totals

,candidate_name,contributor_zip_code,contributors,state,county_name,zip_population,county_fip
0,ADAM CLAYTON POWELL FOR CONGRESS,08901,1,New Jersey,Middlesex County,55223,34023
1,BERNIE,08901,21,New Jersey,Middlesex County,55223,34023
2,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,08901,3,New Jersey,Middlesex County,55223,34023
3,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,08901,1,New Jersey,Middlesex County,55223,34023
4,END CITIZENS UNITED PAC,08901,2,New Jersey,Middlesex County,55223,34023
5,ADAM CLAYTON POWELL FOR CONGRESS,10018,1,New York,New York County,5229,36061
6,BERNIE,10018,10,New York,New York County,5229,36061
7,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,10018,3,New York,New York County,5229,36061
8,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,10018,2,New York,New York County,5229,36061
9,END CITIZENS UNITED PAC,10018,1,New York,New York County,5229,36061


In [306]:
def calculate_per_capita(row):
    if int(row["zip_population"]) == 0: return None
    return round(row["contributors"] * 1000.0 / float(row["zip_population"]), 1)

In [307]:
zip_totals["donors_per_1000_people"] = (
    zip_totals
    .apply(
        calculate_per_capita, 
        axis = 1)
    )


Look at Bernie's top zipcodes

In [308]:
(
    zip_totals
    .loc[ lambda x: x["candidate_name"] == "BERNIE" ]
    .sort_values(
        "contributors",
        ascending = False
    )
    .head(10)
)

,candidate_name,contributor_zip_code,contributors,state,county_name,zip_population,county_fip,donors_per_1000_people
42473,BERNIE,05401,404,Vermont,Chittenden County,28185,50007,14.3
42629,BERNIE,05602,168,Vermont,Washington County,11916,50023,14.1
42427,BERNIE,05301,168,Vermont,Windham County,16820,50025,10.0
12448,BERNIE,98103,148,Washington,King County,45911,53033,3.2
42483,BERNIE,05403,138,Vermont,Chittenden County,17593,50007,7.8
11016,BERNIE,94110,136,California,San Francisco County,69333,06075,2.0
4276,BERNIE,14850,134,New York,Tompkins County,63886,36109,2.1
12532,BERNIE,98115,128,Washington,King County,46206,53033,2.8
78,BERNIE,10025,127,New York,New York County,94600,36061,1.3
662,BERNIE,11215,123,New York,Kings County,63488,36047,1.9


Top zipcodes per capita, excluding Vermont

In [311]:
(
    zip_totals
    .loc[ lambda x: x["candidate_name"] == "BERNIE" ]
    .loc[
        lambda x: (x["zip_population"] >= 1000) & (x["state"] != "Vermont")
    ]
    .sort_values(
        "donors_per_1000_people", 
        ascending = False
    ).head(20)
)

,candidate_name,contributor_zip_code,contributors,state,county_name,zip_population,county_fip,donors_per_1000_people
88514,BERNIE,81131,13,Colorado,Saguache County,1189,08109,10.9
38498,BERNIE,01330,15,Massachusetts,Franklin County,1506,25011,10.0
40190,BERNIE,02535,11,Massachusetts,Dukes County,1177,25007,9.3
38373,BERNIE,01098,10,Massachusetts,Hampshire County,1101,25015,9.1
90030,BERNIE,87010,10,New Mexico,Santa Fe County,1139,35049,8.8
38298,BERNIE,01072,13,Massachusetts,Franklin County,1478,25011,8.8
41123,BERNIE,03280,9,New Hampshire,Sullivan County,1123,33019,8.0
96583,BERNIE,95468,10,California,Mendocino County,1258,06045,7.9
95839,BERNIE,94973,10,California,Marin County,1429,06041,7.0
41204,BERNIE,03466,9,New Hampshire,Cheshire County,1332,33005,6.8


Merge zip_totals with a GeoDataFrame of counties

In [319]:
zip_totals

,candidate_name,contributor_zip_code,contributors,state,county_name,zip_population,county_fip,donors_per_1000_people
0,ADAM CLAYTON POWELL FOR CONGRESS,08901,1,New Jersey,Middlesex County,55223,34023,0.0
1,BERNIE,08901,21,New Jersey,Middlesex County,55223,34023,0.4
2,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,08901,3,New Jersey,Middlesex County,55223,34023,0.1
3,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,08901,1,New Jersey,Middlesex County,55223,34023,0.0
4,END CITIZENS UNITED PAC,08901,2,New Jersey,Middlesex County,55223,34023,0.0
5,ADAM CLAYTON POWELL FOR CONGRESS,10018,1,New York,New York County,5229,36061,0.2
6,BERNIE,10018,10,New York,New York County,5229,36061,1.9
7,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE,10018,3,New York,New York County,5229,36061,0.6
8,DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE,10018,2,New York,New York County,5229,36061,0.4
9,END CITIZENS UNITED PAC,10018,1,New York,New York County,5229,36061,0.2


Why is this happening...

In [327]:
zip_totals.groupby('county_fip')['county_name'].nunique().max()

8

Select for a candidate and counties

In [329]:
bernie = (
    zip_totals
    .loc[
        lambda x: x['candidate_name'] == "BERNIE"
    ]
    .groupby(['county_fip', 'county_name'])
    .agg({
        "contributors": "sum"
    })
    .reset_index()
)

bernie.head()

,county_fip,county_name,contributors
0,01001,Autauga County,1
1,01001,Chilton County,1
2,01001,Elmore County,1
3,01003,Baldwin County,31
4,01007,Bibb County,2


In [330]:
county_geo = (
    gpd
    .read_file("../data/counties-geojson.json")
).merge(
    zip_totals.groupby('',
    left_on = "",
    right_on = "id"
)

county_geo

SyntaxError: invalid syntax (<ipython-input-330-73a5a9e8fbbb>, line 10)

In [ ]:
end = datetime.datetime.now()

d = (end - start)

f"The notebook ran for {round(d.total_seconds() / 60, 2) } minutes"

---

---

---